In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import torchvision.utils as vutils




In [2]:
% load_ext autoreload 
% autoreload 2

In [3]:
from modules_tied import CVAE
from modules_tied import NetD
from modules_tied import Aux
from modules_tied import loss_function

In [4]:
bsz = 100
criterion = nn.BCELoss()


In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=bsz, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=bsz, shuffle=True)


In [6]:
netG = CVAE()
netD = NetD()
#aux = Aux()
print(netG)

CVAE(
  (fc1): Linear(in_features=784, out_features=400, bias=True)
  (fc21): Linear(in_features=400, out_features=20, bias=True)
  (fc22): Linear(in_features=400, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=400, bias=True)
  (fc4): Linear(in_features=400, out_features=784, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (mu_): Sequential(
    (0): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(8, 64, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): Conv2d(64, 20, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (7): ReLU(inplace)
  )
  (logsigma_): Sequential(
    (0): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_

In [7]:
optimizerD = optim.Adam(netD.parameters(), lr=1e-3)
optimizerG = optim.Adam(netG.parameters(), lr=1e-3)
#optimizer_aux = optim.Adam(aux.parameters(), lr=1e-4)

In [8]:
input = torch.FloatTensor(bsz,28,28)
label = torch.FloatTensor(bsz)
real_label=1
fake_label=0
USE_CUDA=1
lamb = 1e-2
l1dist = nn.PairwiseDistance(1)
l2dist = nn.PairwiseDistance(2)
LeakyReLU = nn.LeakyReLU(0)

if(USE_CUDA):
    netG=netG.cuda()
    netD=netD.cuda()
    #aux = aux.cuda()
    #criterion=criterion.cuda()
    input,label=input.cuda(), label.cuda()
    l1dist = l1dist.cuda()
    l2dist = l2dist.cuda()
    LeakyReLU = LeakyReLU.cuda()

In [9]:
def get_direct_gradient_penalty(netD, x, gamma, cuda):
    _,output = netD(x)
    gradOutput = torch.ones(output.size()).cuda() if cuda else torch.ones(output.size())
    
    gradient = torch.autograd.grad(outputs=output, inputs=x, grad_outputs=gradOutput, create_graph=True, retain_graph=True, only_inputs=True)[0]
    gradientPenalty = (gradient.norm(2, dim=1)).mean() * gamma
    
    return gradientPenalty


In [ ]:
from train import run_trainer

In [ ]:
run_trainer(netG,netD,optimizerG,optimizerD,train_loader,bsz)

epoch  0
epoch  0
epoch  0
epoch  0
epoch  0
epoch  0
losses 0 -24.5439539112 359.535509527
e_enc [359.53550952672958]
epoch  1
epoch  1
epoch  1
epoch  1
epoch  1
epoch  1
losses 1 -72.1103717624 308.976175845
e_enc [359.53550952672958, 308.97617584466934]
epoch  2
epoch  2
epoch  2
epoch  2
epoch  2
epoch  2
losses 2 -40.0685885921 257.022022933
e_enc [359.53550952672958, 308.97617584466934, 257.02202293276787]
epoch  3
epoch  3
epoch  3
epoch  3
epoch  3
epoch  3
losses 3 -46.3214931227 272.593817651
e_enc [359.53550952672958, 308.97617584466934, 257.02202293276787, 272.59381765127182]
epoch  4
epoch  4
epoch  4
epoch  4
epoch  4
epoch  4
losses 4 -51.0576077122 266.891194373
e_enc [359.53550952672958, 308.97617584466934, 257.02202293276787, 272.59381765127182, 266.89119437336922]
epoch  5
epoch  5
epoch  5
epoch  5
epoch  5
epoch  5
losses 5 -47.0922481082 252.284244508
e_enc [359.53550952672958, 308.97617584466934, 257.02202293276787, 272.59381765127182, 266.89119437336922, 252.28

In [ ]:
label_ = torch.LongTensor(64,1)

for i in range(64):
    label_[i][0] = 5

onehot_tmp = torch.FloatTensor(64,10)

sample = Variable(torch.randn(64, 20))
onehot_tmp.zero_()
onehot_tmp.scatter_(1,label_,1)
onehot = Variable(onehot_tmp)
    
    
if USE_CUDA==1:
    sample = sample.cuda()
    onehot = onehot.cuda()
    sample = netG.decode_new(torch.cat((sample,onehot),1)).cpu()
    save_image(sample.data.view(64, 1, 28, 28),
                'sample'+ '.png')